# PlayHQ Fixture Scraping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ssardina/tapp-fixture/blob/main/playhq_scrape.ipynb)

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes the following components:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com. This key can be stored in a file `x_api_key.txt` or it will be asked interactively by the notebook otherwise. In many cases, the feature to create new API credentials is disabled for a user and can only be actioned by a Super Administrator role within the Play HQ portal.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.


Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import re
import os
import calendar, datetime
import dtale

import utils
import playhq as phq

## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [2]:
# Change this to import your club's own configuration
from config_bmc_w23 import *
# from config_cba import *

# Set the game date interval scrape
# GAME_DATE_START = datetime.date.today() # by default, any game after today
# GAME_DATE_START = utils.next_day(calendar.SATURDAY)   # start from next game day
GAME_DATE_START = datetime.date(2023, 7, 5) # start on a specific day - REP ROUND 5+

WEEKS = 15   # how many weeks after date start we want to scrape
GAME_DATE_END = GAME_DATE_START + datetime.timedelta(days=WEEKS*7)

###############################################################
# DO NOT CHANGE FROM HERE
###############################################################

# Get nice game date format: Saturday August 06, 2022
GAME_DATE_START_TIMESTAMP = pd.to_datetime(GAME_DATE_START).tz_localize(TIMEZONE)
GAME_DATE_END_TIMESTAMP = pd.to_datetime(GAME_DATE_END).tz_localize(TIMEZONE)

GAME_DATE_START_NAME = utils.pretty_date(GAME_DATE_START_TIMESTAMP)
GAME_DATE_END_NAME = utils.pretty_date(GAME_DATE_END_TIMESTAMP)

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)
if SEASON_ID is None:
    raise SystemExit("ERROR! Please specify either SEASON_ID.")
SEASON_NAME = phq_club.get_season_name(SEASON_ID)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print(f"Season: {SEASON_NAME} (season id: {SEASON_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)
print("Output path:", OUTPUT_PATH)
if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

print("Timezone:", TIMEZONE)
print(f"Game dates: {GAME_DATE_START_NAME} - {GAME_DATE_END_NAME}")
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)

Club name: Brunswick Magic Basketball Club (org. id: 8c4d5431-eaa5-4644-82ac-992abe224b88)
Season: Junior Domestic (season id: cdbe3065-2a32-4c6d-8771-f8fae3fa7611)
X-tenant: bv x-api-key: f5d33c76-f858-49fa-8330-8e0e396219cd
Output path: Brunswick_Magics/2023.01.Winter-23/fixture/
Timezone: Australia/Melbourne
Game dates: Wednesday July 05, 2023 (2023/07/05) - Wednesday October 18, 2023 (2023/10/18)
PlayHQ Club fixture: https://bit.ly/bmbc-w23


## 2. Get upcoming games for club's teams

First, get the teams of the club, sort them based on age group.

In [ ]:
teams_df = phq_club.get_season_teams(SEASON_ID)
teams_df.sort_values('age', ascending=True, inplace=True)
teams_df.reset_index(inplace=True, drop=True)

teams = teams_df['name'].values
print(f"Found {len(teams)} teams:", teams)

# teams_df
# teams_df.query("name == 'Coburg U12 Boys 1'")

# if needed keep only a specific team (e.g., for debugging later)
# teams_df = teams_df.query("id == '115ca2a9-5383-4beb-9703-691006568974'")

Next, extract all games between the dates specified for these teams of the club.

In [ ]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_START_TIMESTAMP, GAME_DATE_END_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print(f'No games between {GAME_DATE_START_NAME} and {GAME_DATE_END_NAME}')

In [ ]:
# dtale.show(upcoming_games_df)
upcoming_games_df

## 3. Convert to TeamApp CSV format

Next, we convert the PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

This process takes time as it processes game per game and even obtains short URL links for each game.

In [ ]:
games_tapps_df = phq_club.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP, game_duration=45)
print("Done computing the games for Teams App")

# find out the game day if there is one day all teams play on only
game_day = None
single_game_day = (games_tapps_df['start_date'].drop_duplicates().size == 1)
if single_game_day:
    game_day = games_tapps_df.iloc[0]['start_date']
    print("All games are in the following day:", utils.pretty_date(game_day))

Inspect how the description of one of the games will look like:

In [ ]:
# games_tapps_df.sample(3)
# dtale.show(games_tapps_df)
games_tapps_df

Keep games after a particular date:

In [ ]:
import datetime

START_DATE = datetime.date(2023, 7, 21)

games_tapps_df[games_tapps_df['start_date'] > START_DATE]

games_tapps_df = games_tapps_df[games_tapps_df['start_date'] > START_DATE]

games_tapps_df

In [ ]:
# Inspect description game of one team
team = "20.2 Boys"
# team = "U12 Boys Purple"

print("Description for:", team)
print(games_tapps_df.query("team_name == @team")['description'].values[0])

## 4. Append BYE games (if necessary)

We generate BYE entries for TeamsAPP ***only*** if the games are played all in the same day.

In [ ]:
bye_teams= False    # assume no bye games

if game_day is not None:
    game_day = games_tapps_df.iloc[0]['start_date']

    # Extract the date of the round
    # date = games_tapps_df.iloc[1]['start_date']
    print(f"Extract BYE games for games on {utils.pretty_date(game_day)}")

    playing_teams = upcoming_games_df['team_id'].tolist()
    bye_teams = teams_df.loc[~teams_df['id'].isin(playing_teams)]['name'].tolist()
    bye_teams = list(map(lambda x: re.search("U.*", x).group(0), bye_teams))

    if bye_teams:
        games_bye_df = phq_club.build_teamsapp_bye_schedule(bye_teams, game_day, DESC_BYE_TAPP)
        print(f"Bye teams ({len(bye_teams)}): ", bye_teams)

        games_tapps_df = pd.concat([games_tapps_df, games_bye_df])
        games_tapps_df.drop_duplicates(inplace=True)
        games_tapps_df.reset_index(inplace=True, drop=True)
    else:
        print("No BYE games this round...")
else:
    print("Games obtained are not on the same day, not computing BYE games...")

(bye_teams and games_bye_df)

## 5. Final review

Finally, we report the games to be written into Schedule CSV file and **CHECKING THAT ALL IS GOOD TO GO!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [ ]:
games_tapps_df.columns
games_tapps_df[['event_name', 'team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
# games_tapps_df

We stop the execution here if we are running all Jupyter notebook.

In [ ]:
raise SystemExit("Stop right there! Continue below to produce the CSV file if needed.")

## 6. Save to CSV file for Teams App import

OK we are ready to import into Teams App.

### 6.2. Check changes with previous saves

If the schedule was generated before, check if the new one differs with the one saved already.

First, let us define the files that we will save to disk.

In [ ]:
now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y_%m_%d-%H:%M:%S")

id_file = now_str
if game_day is not None:    # there is one date for all games!
    id_file = utils.compact_date(game_day)

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{id_file}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{id_file}.pkl")
file_games_tapps = os.path.join(OUTPUT_PATH, f"games_tapps_df-{id_file}.pkl")

print("Files to save:")
print(file_csv)
print(file_upcoming_pkl)
print(file_games_tapps)

if not os.path.exists(OUTPUT_PATH):
    raise SystemExit("ERROR! Output path {OUTPUT_PATH} is missing! Please create or link that path correctly to save data.")

Next, let's check if there was a saved file for the upcoming game day.

In [ ]:
cols = ['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']

changed_games_df = None
if os.path.exists(file_games_tapps):
    print("There was already a schedule saved, recovering it to compare...")
    old_games_tapps_df = pd.read_pickle(file_games_tapps)

    teams_changed = pd.concat([games_tapps_df[cols], old_games_tapps_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()
    print("Teams whose games have changed (updated, new, dropped):", teams_changed)

    old_games_df = old_games_tapps_df[cols].query("team_name in @teams_changed")
    new_games_df = games_tapps_df[cols].query("team_name in @teams_changed")
    changed_games_df = new_games_df.merge(old_games_df, how="inner", on="team_name", suffixes=('_new', '_old'))
else:
    print("No previous schedule saved")

# Show changes if any...
changed_games_df

### 5.3. Write a TeamAPP Schedule CSV & Datafarmes Pickles

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [ ]:
import shutil

print('Saving TeamAPP schedule CSV file and Dataframes with id:', id_file)
for f in [file_csv, file_upcoming_pkl, file_games_tapps]:
  if os.path.exists(f):
    print("Backup file", f)
    shutil.copy(f, f + ".bak")

print('Saving CSV TeamApp schedule:', file_csv)
games_tapps_df.to_csv(file_csv, index=False)

print('Saving dataframe pickle:', file_upcoming_pkl)
upcoming_games_df.to_pickle(file_upcoming_pkl)
print('Saving dataframe pickle:', file_games_tapps)
games_tapps_df.to_pickle(file_games_tapps)

print(f"Finished saving CSV and DATA-FRAMNE files: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

# ------------ END FIXTURE PUBLISHING ------------

### Check a particular team

In [ ]:
team = "U10 Girls Gold"

print(games_tapps_df.query("team_name == @team")['description'].values[0])
games_tapps_df.query("team_name == @team")[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
